Ok so now I have tried PCA with X results. 



There are several things I did try. 
The first was feature engineering. 

I read this paper: 
http://www.deepscn.com/pdfs/Pao%20and%20Takefji_1992.pdf

In a nutshell, it says that if you add more features derived from other features.
If you read it, it's a bit of a mystery how it works. But let's give it a shot and see what it does. 

Normally, I'd also  would based on domain knowledge
(for example, if you're trying to predict power usage, you might create another feature based on temperature and humidity), but in this case I didn't really have a lot of context for the dataset beyond "this is an anti-fraud dataset" so I decided to just used the tan function to help me.

To keep it simple and easier to do compare, I also used the same architecture as prior with only the input layer changed. 


In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
import numpy as np

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict

import copy
import pandas as pd

In [6]:
#https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
#https://medium.com/@shashikachamod4u/excel-csv-to-pytorch-dataset-def496b6bcc1
#Learning rate modifier downstairs

#https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html


#seed 0 is a good one!
#https://stackoverflow.com/questions/19306976/python-shuffling-with-a-parameter-to-get-the-same-result


csv = pd.read_csv("class 77 synth.csv", sep=';' )
csv = shuffle(csv)

pre_full = csv
pre_train=csv[0:2500]
pre_test=csv[2500:5000]

In [7]:
csv

,X2,X3,X4,X5,X6,TANX2,TanX3,Tanx4,TanX5,TanX6,class
2098,-0.936030,0.855527,0.006320,-0.560525,0.044400,-1.357883,1.151101,0.006320,-0.627681,0.044429,1
3189,0.490733,0.347759,0.165601,0.687637,0.163640,0.534330,0.362490,0.167131,0.821372,0.165117,1
1118,0.608384,0.270552,-0.356786,-0.830551,-0.671067,0.696516,0.277353,-0.372738,-1.094644,-0.793993,1
3558,-0.098200,-0.597729,-0.407794,-0.692466,-0.086400,-0.098517,-0.680808,-0.432011,-0.829491,-0.086616,1
2636,0.511986,0.107689,0.336475,0.409854,0.092500,0.561969,0.108108,0.349775,0.434458,0.092765,1
...,...,...,...,...,...,...,...,...,...,...,...
2260,0.074900,0.018700,0.189390,0.137721,0.656542,0.075040,0.018702,0.191687,0.138599,0.770579,1
1995,0.395661,-0.718361,-0.299357,-0.888328,0.629872,0.417688,-0.874172,-0.308632,-1.230388,0.728919,1
4901,0.670007,-0.170011,-0.730044,0.465083,-0.245571,0.792266,-0.171668,-0.894998,0.501796,-0.250630,1
2022,-0.198517,0.591385,0.450856,-0.198618,0.294775,-0.201167,0.671563,0.484111,-0.201271,0.303620,1


In [8]:
#turn CSV panda into numpy array
csv_np = csv.values
pre_train_np = pre_train.values
pre_test_np = pre_test.values

In [9]:
class csvDataset(Dataset):
    def __init__(self,file):
        file_out = file #pd.read_csv(file)
        x=file_out.iloc[:,0:-1].values
        y=file_out.iloc[:,-1:].values

        sc = StandardScaler()
        x_train = sc.fit_transform(x)
        y_train = y
        
        self.X_train = torch.tensor(x_train,dtype = torch.float32)
        self.y_train = torch.tensor(y_train)
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self,idx):
        return self.X_train[idx].float(),self.y_train[idx].float()
    
        

In [10]:
full = csvDataset(pre_full)
train = csvDataset(pre_train)
test = csvDataset(pre_test)

In [11]:
batch_size =500
full_dataloader = DataLoader(full, batch_size=batch_size)
train_dataloader = DataLoader(train, batch_size=batch_size)
test_dataloader = DataLoader(test, batch_size=batch_size)


If you want to load

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [12]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 12),
            nn.Tanh(),
            nn.Linear(12, 18),
            nn.Tanh(),
            nn.Linear(18, 2),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

#https://discuss.pytorch.org/t/reset-model-weights/19180
# Neural network model |---> Void
#Usage : model.apply(weight_init)
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight.data)
        

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10, out_features=12, bias=True)
    (1): Tanh()
    (2): Linear(in_features=12, out_features=18, bias=True)
    (3): Tanh()
    (4): Linear(in_features=18, out_features=2, bias=True)
    (5): Tanh()
  )
)


https://pytorch.org/docs/stable/nn.html#loss-functions


https://pytorch.org/docs/stable/optim.html

https://ruder.io/optimizing-gradient-descent/index.html#gradientdescentvariants

https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

https://analyticsindiamag.com/ultimate-guide-to-pytorch-optimizers/

In [13]:
#https://stackoverflow.com/questions/56435961/how-to-access-the-network-weights-while-using-pytorch-nn-sequential
#model.linear_relu_stack[0].weight = nn.Parameter(model.linear_relu_stack[0].weight*2)

In [14]:
loss_fn = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False) 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
#This is the testing function: 
# it tests for  and 
# returns accuracy

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print(size)
    num_batches = len(dataloader)
    #model.eval() will notify all your layers that you are in eval mode
    model.eval()
    test_loss, correct = 0, 0
    #torch.no_grad() impacts the autograd engine and deactivate it. It will reduce memory usage and speed up computations but you won’t be able to backprop
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
                
            final_pred = pred.argmax(1)            
            correct += torch.eq(final_pred,y.flatten()).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

https://medium.com/mini-distill/effect-of-batch-size-on-training-dynamics-21c14f7a716e

Learning rate modifier:

https://stackoverflow.com/questions/48324152/pytorch-how-to-change-the-learning-rate-of-an-optimizer-at-any-given-moment-no

I did one extra thing here: 
    
I changed the learning rate so that it would decrease exponentially fast based on whether or not the last iteration made any improvement up to a certain amount of steps before resetting. 

It works as an exploration bonus.The big idea here is to be able to "leap" through local minima. 
Think of it as a soldier sweeping a minefield with his metal detector before taking a step in the safest direction.
Repeat ad nauseum or until safety is reached.


In [17]:
end_epoch = 500

#Usage:
#n_epoch=number of epoch, start with 0
#end_epoch = at what time do you want to end the epoch? 
#previous = the best score (in % ) up to date
#step_size

def test_train(n_epoch, end_epoch, previous_best,step_size_idx,very_best,best_model):
    print(f"Epoch {n_epoch}\n-------------------------------")
    print(f"Previous best:{previous_best}")
    print(f"step size index:{step_size_idx}")
    
    lr = 1/2**(step_size_idx%30)
    print(f"learning rate: {lr}")
#    optimizer = torch.optim.Adam(model.parameters(), lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    for g in optimizer.param_groups:
        g['lr'] = lr

    train(train_dataloader, model, loss_fn, optimizer)
    score = test(test_dataloader, model, loss_fn)
    
    if very_best < score:
        best_model = copy.deepcopy(model)
        very_best = score

    if score <= previous_best:
        step_size_idx += 1 
        best = score
    else:
        best = score        
    
    if n_epoch >= end_epoch:
        print("done!")
        return best_model
    else:
        return test_train(n_epoch+1,end_epoch, best, step_size_idx,very_best,best_model)



Lets take this baby for a test drive: 

In [64]:
torch.manual_seed(100)

best_model = NeuralNetwork().to(device)

#best_model.apply(weights_init)
model_1 =test_train(0,end_epoch,0,0,0,best_model)

Epoch 0
-------------------------------
Previous best:0
step size index:0
learning rate: 1.0
loss: 0.945484  [    0/ 2500]
2500
Test Error: 
 Accuracy: 16.7%, Avg loss: 0.015076 

Epoch 1
-------------------------------
Previous best:0.1672
step size index:0
learning rate: 1.0
loss: 0.014253  [    0/ 2500]
2500
Test Error: 
 Accuracy: 26.1%, Avg loss: 0.012355 

Epoch 2
-------------------------------
Previous best:0.2612
step size index:0
learning rate: 1.0
loss: 0.011512  [    0/ 2500]
2500
Test Error: 
 Accuracy: 33.9%, Avg loss: 0.011174 

Epoch 3
-------------------------------
Previous best:0.3388
step size index:0
learning rate: 1.0
loss: 0.010321  [    0/ 2500]
2500
Test Error: 
 Accuracy: 41.1%, Avg loss: 0.010519 

Epoch 4
-------------------------------
Previous best:0.4108
step size index:0
learning rate: 1.0
loss: 0.009659  [    0/ 2500]
2500
Test Error: 
 Accuracy: 44.6%, Avg loss: 0.010104 

Epoch 5
-------------------------------
Previous best:0.4456
step size index:0
l

/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([500, 1])) that is different to the input size (torch.Size([500, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 0.008444  [    0/ 2500]
2500
Test Error: 
 Accuracy: 53.2%, Avg loss: 0.009225 

Epoch 10
-------------------------------
Previous best:0.5324
step size index:0
learning rate: 1.0
loss: 0.008338  [    0/ 2500]
2500
Test Error: 
 Accuracy: 54.6%, Avg loss: 0.009140 

Epoch 11
-------------------------------
Previous best:0.546
step size index:0
learning rate: 1.0
loss: 0.008249  [    0/ 2500]
2500
Test Error: 
 Accuracy: 55.5%, Avg loss: 0.009068 

Epoch 12
-------------------------------
Previous best:0.5548
step size index:0
learning rate: 1.0
loss: 0.008172  [    0/ 2500]
2500
Test Error: 
 Accuracy: 56.4%, Avg loss: 0.009006 

Epoch 13
-------------------------------
Previous best:0.5636
step size index:0
learning rate: 1.0
loss: 0.008104  [    0/ 2500]
2500
Test Error: 
 Accuracy: 57.3%, Avg loss: 0.008951 

Epoch 14
-------------------------------
Previous best:0.5728
step size index:0
learning rate: 1.0
loss: 0.008043  [    0/ 2500]
2500
Test Error: 
 Accuracy: 57.9%, Avg l

Very good! High accuracy but lower avg loss then the previous approach. Let's save it:

In [65]:
torch.save(model_1.state_dict(), "model_1.pth")

In [33]:
model_1 = NeuralNetwork().to(device)
model_1.load_state_dict(torch.load("model_1.pth"))

<All keys matched successfully>

In [19]:
#model_1.linear_relu_stack[0].weight

Now here's the thing: the performance of a neural network can be is highly dependent on the seed with which its weights were initialized.

You can think of the parameter space as a valley with the random weight initializer being Herculus throwing a discus in a hilly plain. 
If the discuss lands in a spot that's proper for SGD to do its job, or else it might just be stuck at a local maxima for a very very long time. 

To illustrate:

In [75]:
torch.manual_seed(5)

model = NeuralNetwork().to(device)
best_model = NeuralNetwork().to(device)

#best_model.apply(weights_init)
model_2 =test_train(0,end_epoch,0,0,0,best_model)

Epoch 0
-------------------------------
Previous best:0
step size index:0
learning rate: 1.0
loss: 1.089532  [    0/ 2500]
2500
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.088265 

Epoch 1
-------------------------------
Previous best:0.2656
step size index:0
learning rate: 1.0
loss: 1.089532  [    0/ 2500]
2500
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.088265 

Epoch 2
-------------------------------
Previous best:0.2656
step size index:1
learning rate: 0.5
loss: 1.089532  [    0/ 2500]
2500
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.088265 

Epoch 3
-------------------------------
Previous best:0.2656
step size index:2
learning rate: 0.25
loss: 1.089532  [    0/ 2500]
2500
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.088265 

Epoch 4
-------------------------------
Previous best:0.2656
step size index:3
learning rate: 0.125
loss: 1.089532  [    0/ 2500]
2500
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.088265 

Epoch 5
-------------------------------
Previous best:0.2656
step size index:

No where near as good. 

Right so this experiment didn't lead anywhere.
Let's try another random seed:

In [76]:
torch.manual_seed(200)

model = NeuralNetwork().to(device)
best_model = NeuralNetwork().to(device)

#best_model.apply(weights_init)
model_3 =test_train(0,end_epoch,0,0,0,best_model)

Epoch 0
-------------------------------
Previous best:0
step size index:0
learning rate: 1.0
loss: 0.657311  [    0/ 2500]
2500
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.657997 

Epoch 1
-------------------------------
Previous best:0.7812
step size index:0
learning rate: 1.0
loss: 0.657311  [    0/ 2500]
2500
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.657997 

Epoch 2
-------------------------------
Previous best:0.7812
step size index:1
learning rate: 0.5
loss: 0.657311  [    0/ 2500]
2500
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.657997 

Epoch 3
-------------------------------
Previous best:0.7812
step size index:2
learning rate: 0.25
loss: 0.657311  [    0/ 2500]
2500
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.657997 

Epoch 4
-------------------------------
Previous best:0.7812
step size index:3
learning rate: 0.125
loss: 0.657311  [    0/ 2500]
2500
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.657997 

Epoch 5
-------------------------------
Previous best:0.7812
step size index:

Better, but can we get something closer to what we want? 
Let's try something else:

In [101]:
torch.manual_seed(0)

model = NeuralNetwork().to(device)
best_model = NeuralNetwork().to(device)

best_model.apply(weights_init)
model_4 =test_train(0,end_epoch,0,0,0,best_model)

Epoch 0
-------------------------------
Previous best:0
step size index:0
learning rate: 1.0
loss: 1.029352  [    0/ 2500]
2500
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.019963 

Epoch 1
-------------------------------
Previous best:0.9868
step size index:0
learning rate: 1.0
loss: 1.029352  [    0/ 2500]
2500
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.019963 

Epoch 2
-------------------------------
Previous best:0.9868
step size index:1
learning rate: 0.5
loss: 1.029352  [    0/ 2500]
2500
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.019963 

Epoch 3
-------------------------------
Previous best:0.9868
step size index:2
learning rate: 0.25
loss: 1.029352  [    0/ 2500]
2500
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.019963 

Epoch 4
-------------------------------
Previous best:0.9868
step size index:3
learning rate: 0.125
loss: 1.029352  [    0/ 2500]
2500
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.019963 

Epoch 5
-------------------------------
Previous best:0.9868
step size index:

In [105]:
torch.save(model_4.state_dict(), "model_4.pth")

In [19]:
torch.manual_seed(0)

model = NeuralNetwork().to(device)
best_model = NeuralNetwork().to(device)

best_model.apply(weights_init)
model_5 =test_train(0,end_epoch,0,0,0,best_model)

Epoch 0
-------------------------------
Previous best:0
step size index:0
learning rate: 1.0


/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([500, 1])) that is different to the input size (torch.Size([500, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 1.024742  [    0/ 2500]
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.022875 

Epoch 1
-------------------------------
Previous best:0.9896
step size index:0
learning rate: 1.0
loss: 1.024742  [    0/ 2500]
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.022875 

Epoch 2
-------------------------------
Previous best:0.9896
step size index:1
learning rate: 0.5
loss: 1.024742  [    0/ 2500]
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.022875 

Epoch 3
-------------------------------
Previous best:0.9896
step size index:2
learning rate: 0.25
loss: 1.024742  [    0/ 2500]
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.022875 

Epoch 4
-------------------------------
Previous best:0.9896
step size index:3
learning rate: 0.125
loss: 1.024742  [    0/ 2500]
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.022875 

Epoch 5
-------------------------------
Previous best:0.9896
step size index:4
learning rate: 0.0625
loss: 1.024742  [    0/ 2500]
2500
Test Error: 
 Accuracy: 99.0%, Avg

OK well for whatever reason, it looks like I lucked out!

To reuse the minesweeper analogy, looks like our unlucky soldier landed on an untouched patch of grass with nymphs, rose water, and coconuts, and smartly figured out that everything else is a battlefield. It looks like he doesn't want to take even a tiny step in any direction. 

There might be a bug somewhere giving him depression, but who knows?


In [20]:
torch.save(model_5.state_dict(), "model_5.pth")

In [24]:
#for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor])

We can take a look at what the tensor list looks like:

In [28]:
tensor_list = [tensor for tensor in model_1.state_dict()]

In [29]:
tensor_list

['linear_relu_stack.0.weight',
 'linear_relu_stack.0.bias',
 'linear_relu_stack.2.weight',
 'linear_relu_stack.2.bias',
 'linear_relu_stack.4.weight',
 'linear_relu_stack.4.bias']

OK so now we have two models. Model_1 and model_5.

The first has so-so accuracy and average error, and the second has .

Can we do better?
Yes.  Let's write a function that will give us weights somewhere in between the two extremes:


In [34]:
test_model = NeuralNetwork().to(device)

def linear_combo_dic(model_a,model_b,t):
    od = OrderedDict()
    for tensor in model_a.state_dict():
        od[tensor]= model_a.state_dict()[tensor]*t + (1-t)*model_b.state_dict()[tensor]
    return od


<All keys matched successfully>

^ TLDR: Given two matrices of weights, A and B, and a number t in [0,1], 

the function above returns A*t + (1-t)*B

In [ ]:
Now that we have 

In [38]:
best_score=0
best_t=-1

for t in np.linspace(0,1,1000):
    print(t)
    od = linear_combo_dic(model_1,model_5,t)
    test_model.load_state_dict(od)
    
    score = test(test_dataloader, test_model, loss_fn)
    
    if best_score <score:
        best_score=score
        best_t=t
        

print(best_score)
print(best_t)
    


0.0
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.022875 

0.001001001001001001
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.020127 

0.002002002002002002
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.017348 

0.003003003003003003
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.014608 

0.004004004004004004
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.011890 

0.005005005005005005
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.009105 

0.006006006006006006
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.006418 

0.007007007007007007
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.003655 

0.008008008008008008
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 1.000944 

0.009009009009009009
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.998048 

0.01001001001001001
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.995375 

0.011011011011011011
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.992610 

0.012012012012012012
2500
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.989791 

0.01

So there you have it.  

It's important to note that a 99% accuracy rate is suspect is likely already overfitting. 
But it's just to say that it's still possible to make improvements via just checking around the area manually.